# Example: QAOA program

This tutorial will be demonstation of creating QAOA Program as well as migration guide on how you can replicate IBM Quantum QAOA custom runtime program.

Let's first get information on what is QAOA runtime program and what inputs and outputs it has. We will not be implementing full set of input/outputs, but we will cover most important ones. Later on we can recover full functionality if needed.

**Description**: Qiskit Runtime QAOA program.




**Inputs**:

| name | type | description |
| ---- | ---- | ----------- |
|initial_point|[array,string]|Initial parameters of the ansatz. Can be an array or the string ``'random'`` to choose random initial parameters. The type must be numpy.ndarray or str.|
|operator|object|The cost Hamiltonian, consisting of Pauli I and Z operators, whose smallest eigenvalue we're trying to find. The type must be a PauliSumOp.|
|optimization_level|integer|The optimization level to run if the swap strategies are not used. This value is 1 by default. This is an integer.|
|optimizer|object|The classical optimizer used to update the parameters in each iteration. Per default, SPSA with automatic calibration of the learning rate is used. The type must be a qiskit.algorithms.optimizers.Optimizer.|
|reps|integer|The number of QAOA repetitions, i.e. the QAOA depth typically labeled p. This value defaults to 1. This is an integer.|

**Return values**

| name | type | description |
| ---- | ---- | ----------- |
|cost_function_evals|number|The number of cost function (energy) evaluations. This is an integer.|
|eigenstate|object|The square root of sampling probabilities for each computational basis state of the circuit with optimal parameters.|
|eigenvalue|number|The estimated smallest eigenvalue.|
|optimal_parameters|null|Not supported at the moment, therefore ``None``.|
|optimal_point|array|The optimal parameter values found during the optimization.|
|optimal_value|number|The smallest value found during the optimization. Equal to the ``eigenvalue`` attribute.|
|optimizer_time|number|The total time taken by the optimizer.|


We will also add optional `QiskitRuntimeService` as an argument to use that to access real devices.


With that information we can start drafting our program implementation in `qaoa.py` file.

What our program should do:
1. parse input arguments
2. create run_qaoa function that accepts estimator instance, creates VQE and runs calculation
3. decide which sampler to use and run vqe
    - if runtime service was passed then create a session and run `run_qaoa` function
    - if runtime service was not passed then use stantard qiskit sampler
4. save results from qaoa

Roughly our QAOA program will look like this

```python
# qaoa.py

import ...

def run_qaoa(sampler: BaseSampler, optimizer: Optimizer, reps: int, operator: PauliSumOp):
    qaoa = QAOA(sampler, optimizer, reps=reps)
    return qaoa.compute_minimum_eigenvalue(operator)


arguments = get_arguments()
service = arguments.get("service")
operator = arguments.get("operator")
initial_point = arguments.get("initial_point")
reps = arguments.get("reps", 1)
optimizer = ...
...
if service is not None:
    # if we have service we need to open a session and create sampler
    service = arguments.get("service")        
    backend = arguments.get("backend", "ibmq_qasm_simulator")
    with Session(service=service, backend=backend) as session:
        sampler = Sampler(session=session, options=options)
        result = run_qaoa(sampler, optimizer, reps, operator)
else:
    # if we do not have a service let's use standart local sampler
    sampler = QiskitSampler()
    result = run_qaoa(sampler, optimizer, reps, operator)

save_result({
    "cost_function_evals": result.cost_function_evals,
    "eigenstate": result.eigenstate,
    "eigenvalue": result.eigenvalue,
    "optimal_parameters": list(result.optimal_parameters.values()),
    "optimal_point": result.optimal_point.tolist(),
    "optimal_value": result.optimal_value,
    "optimizer_time": result.optimizer_time
})
```

At this point we have our program implemented. Now we need to actually run it. But before let's prepare input arguments from our QAOA program.

In [2]:
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService

USE_RUNTIME_SERVICE = False

service = None
backend = "ibmq_qasm_simulator"
if USE_RUNTIME_SERVICE:
    service = QiskitRuntimeService()

input_arguments = {
    "initial_point": None,
    "operator": SparsePauliOp("ZIIZ"),
    "optimization_level": 1,
    "optimizer": "cobyla",
    "reps": 2,
    "service": service,
    "backend": backend
}
input_arguments

{'initial_point': None,
 'operator': SparsePauliOp(['ZIIZ'],
               coeffs=[1.+0.j]),
 'optimization_level': 1,
 'optimizer': 'cobyla',
 'reps': 2,
 'service': None,
 'backend': 'ibmq_qasm_simulator'}

In [3]:
from quantum_serverless import QuantumServerless, GatewayProvider
import os

In [4]:
provider = GatewayProvider(
    username="user",
    password="password123",
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

In [5]:
from quantum_serverless import Program

program = Program(
    title="QAOA",
    entrypoint="qaoa.py",
    working_dir="./source_files/qaoa/"
)

job = serverless.run_program(program, arguments=input_arguments)
job

<Job | 8cd76a64-64ea-468f-a295-114dcc5abd2a>

In [6]:
job.status()

'SUCCEEDED'

In [7]:
job.result()

{'cost_function_evals': 56,
 'eigenstate': {'0': 2.6425405e-09,
  '1': 0.1249999973574594,
  '2': 2.6425405e-09,
  '3': 0.1249999973574594,
  '4': 2.6425405e-09,
  '5': 0.1249999973574594,
  '6': 2.6425405e-09,
  '7': 0.1249999973574594,
  '8': 0.1249999973574594,
  '9': 2.6425405e-09,
  '10': 0.1249999973574594,
  '11': 2.6425405e-09,
  '12': 0.1249999973574594,
  '13': 2.6425405e-09,
  '14': 0.1249999973574594,
  '15': 2.6425405e-09},
 'eigenvalue': -0.9999999577193509,
 'optimal_parameters': [-4.261262774122553,
  -1.7722068719193094,
  2.7455687782210543,
  0.6701009936251439],
 'optimal_point': [-4.261262774122553,
  -1.7722068719193094,
  2.7455687782210543,
  0.6701009936251439],
 'optimal_value': -0.9999999577193509,
 'optimizer_time': 0.46456098556518555}